In [1]:

import pandas as pd
import json
from pkg_resources import resource_filename
import numpy as np

In [2]:
tounament=pd.read_csv('/Users/sebas/IDrive-Sync/MetaLocalLLC/RL-Bots/Tournament-gym/Data/MNCAATourneyCompactResults.csv')
seed=pd.read_csv('/Users/sebas/IDrive-Sync/MetaLocalLLC/RL-Bots/Tournament-gym/Data/MNCAATourneySeeds.csv')
rounds=pd.read_csv('/Users/sebas/IDrive-Sync/MetaLocalLLC/RL-Bots/Tournament-gym/Data/MNCAATourneySeedRoundSlots.csv')
slots=pd.read_csv('/Users/sebas/IDrive-Sync/MetaLocalLLC/RL-Bots/Tournament-gym/Data/MNCAATourneySlots.csv')
teams=pd.read_csv('/Users/sebas/IDrive-Sync/MetaLocalLLC/RL-Bots/Tournament-gym/Data/MTeams.csv')
regions=pd.read_csv('/Users/sebas/IDrive-Sync/MetaLocalLLC/RL-Bots/Tournament-gym/Data/MSeasons.csv')
teams=teams[['TeamID','TeamName']]

In [3]:
r1=rounds.rename(columns={'EarlyDayNum':'Daynum'})
r2=rounds.rename(columns={'LateDayNum':'Daynum'})
double_rounds=pd.concat([r1,r2],axis=0)
double_rounds

,Seed,GameRound,GameSlot,Daynum,LateDayNum,EarlyDayNum
0,W01,1,R1W1,136,137.0,NaN
1,W01,2,R2W1,138,139.0,NaN
2,W01,3,R3W1,143,144.0,NaN
3,W01,4,R4W1,145,146.0,NaN
4,W01,5,R5WX,152,152.0,NaN
...,...,...,...,...,...,...
715,Z16b,2,R2Z1,139,NaN,138.0
716,Z16b,3,R3Z1,144,NaN,143.0
717,Z16b,4,R4Z1,146,NaN,145.0
718,Z16b,5,R5YZ,152,NaN,152.0


In [4]:
display(seed),display(rounds),display(tounament),display(teams)

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374
...,...,...,...
2485,2023,Z12,1433
2486,2023,Z13,1233
2487,2023,Z14,1213
2488,2023,Z15,1421


,Seed,GameRound,GameSlot,EarlyDayNum,LateDayNum
0,W01,1,R1W1,136,137
1,W01,2,R2W1,138,139
2,W01,3,R3W1,143,144
3,W01,4,R4W1,145,146
4,W01,5,R5WX,152,152
...,...,...,...,...,...
715,Z16b,2,R2Z1,138,139
716,Z16b,3,R3Z1,143,144
717,Z16b,4,R4Z1,145,146
718,Z16b,5,R5YZ,152,152


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0
...,...,...,...,...,...,...,...,...
2446,2023,146,1274,88,1400,81,N,0
2447,2023,146,1361,57,1166,56,N,0
2448,2023,152,1163,72,1274,59,N,0
2449,2023,152,1361,72,1194,71,N,0


,TeamID,TeamName
0,1101,Abilene Chr
1,1102,Air Force
2,1103,Akron
3,1104,Alabama
4,1105,Alabama A&M
...,...,...
373,1474,Queens NC
374,1475,Southern Indiana
375,1476,Stonehill
376,1477,TX A&M Commerce


(None, None, None, None)

In [5]:
#change the day num to round
round_map={134:0,
          135:0,

          136:1,
          137:1,
        #   140:1,

          138:2,
          139:2,

          143:3,
          144:3,

          145:4,
          146:4,          
        #   147:4,
        #   148:4,            

          152:5,
          
          154:6}

In [6]:
#Standardize the Regions
region_map={'South':'South',
 'MidWest':'Midwest',
 'East':'East',
 'West':'West',
 
 'Southeast':'South',
 'Southwest':'Midwest',

 'Albuquerque':'West',
 'Chicago':'Midwest',
 'Austin':'South',
 'Syracuse':'East',

 'NA1':'East',
 'NA2':'Midwest',
 'NA4':'West',
 'NA3':'South',

 'Oakland':'West',
 'Atlanta':'South',
 'WashingtonDC':'East',
 'Minneapolis':'Midwest',
 'EastRutherford':'East',

 'StLouis':'Midwest',
 'Atlanta':'South',
 'Phoenix':'West',
}

In [7]:
region_to_num={'W':1,
               'X':2,
               'Y':3,
               'Z':4}


In [8]:
regions.filter(like='Region').columns
all_regions=[]

regions.columns=[c.replace('Region','') for c in regions.columns]
seed_region_map=regions.set_index('Season')[['W','X','Y','Z']].T.to_dict()
for season in seed_region_map:
    seed_region_map[season]={i:a for a,i in seed_region_map[season].items()}


In [9]:
seeds=pd.merge(seed,teams,left_on='TeamID',right_on='TeamID',how='left')

seeds['Region']=seeds['Seed'].str[0]


In [10]:
region_to_region={'W':'East',
               'X':'Midwest',
               'Y':'South',
               'Z':'West'}

In [11]:
seasons=seeds.Season.unique()
for season in seasons:
    seeds.loc[seeds.Season==season,'Region_']=seeds.loc[seeds.Season==season,'Region'].replace(seed_region_map[season])
seeds['RegionNum']=seeds['Region'].replace(region_to_num)


seed_df=seeds
seed_df

,Season,Seed,TeamID,TeamName,Region,Region_,RegionNum
0,1985,W01,1207,Georgetown,W,W,1
1,1985,W02,1210,Georgia Tech,W,W,1
2,1985,W03,1228,Illinois,W,W,1
3,1985,W04,1260,Loyola-Chicago,W,W,1
4,1985,W05,1374,SMU,W,W,1
...,...,...,...,...,...,...,...
2485,2023,Z12,1433,VCU,Z,Z,4
2486,2023,Z13,1233,Iona,Z,Z,4
2487,2023,Z14,1213,Grand Canyon,Z,Z,4
2488,2023,Z15,1421,UNC Asheville,Z,Z,4


In [12]:
kaggle_names=sorted(teams['TeamName'].to_list())


In [13]:
tournament_df=pd.merge(tounament,teams,left_on=['WTeamID'],right_on=['TeamID'],how='left')
tournament_df=tournament_df.rename(columns={'TeamName':'WTeam'})
tournament_df=tournament_df.drop('TeamID',axis=1)
########
tournament_df=pd.merge(tournament_df,teams,left_on=['LTeamID'],right_on=['TeamID'],how='left')
tournament_df=tournament_df.rename(columns={'TeamName':'LTeam'})
tournament_df['Round']=tournament_df['DayNum'].replace(round_map)
########
tournament_df=tournament_df.drop(['TeamID'],axis=1)
tournament_df.columns=[C.title() for C in tournament_df.columns]


In [14]:
#change the day num to round
round_map_2021={
140:3}
tournament_df.loc[tournament_df.Season==2021,'Round']=tournament_df.loc[tournament_df.Season==2021,'Daynum'].replace(round_map_2021)

In [15]:
## merge Seeds with the teams
tourn_region=pd.merge(tournament_df,seeds[['TeamName','Seed','Season','Region','RegionNum']],
left_on=['Season','Wteam'],
right_on=['Season','TeamName'],
how='left'
)
tourn_region=tourn_region.drop(['TeamName','Wloc','Numot'],axis=1)
tourn_region=tourn_region.rename(columns={'Seed':'Wseed'})


tourn_region=pd.merge(tourn_region,seeds[['Season','TeamName','Seed']],
left_on=['Season','Lteam'],
right_on=['Season','TeamName'],
how='left'
)

tourn_region=tourn_region.rename(columns={'Seed':'Lseed'})
tourn_region=tourn_region.drop('TeamName',axis=1)


In [16]:
r=1
print(tourn_region[(tourn_region.Season==2021)].shape[0])

tourn2021=tourn_region[(tourn_region.Season==2021)]
add_game=tourn2021.loc[[tourn2021.index[0]]].copy()

add_game.Season=2021
add_game.Daynum=138
add_game.Wteam='Oregon'
add_game.Wseed='X07'
add_game.Wscore=1

add_game.Lteam='VCU'
add_game.Lseed='X10'
add_game.Lscore=0

add_game.Region='X'
add_game.RegionNum=2
add_game.Round=1

tourn_region=pd.concat([tourn_region,add_game],axis=0,ignore_index=True)
tourn_region,add_game

66


(      Season  Daynum  Wteamid  Wscore  Lteamid  Lscore         Wteam  \
 0       1985     136     1116      63     1234      54      Arkansas   
 1       1985     136     1120      59     1345      58        Auburn   
 2       1985     136     1207      68     1250      43    Georgetown   
 3       1985     136     1229      58     1425      55   Illinois St   
 4       1985     136     1242      49     1325      38        Kansas   
 ...      ...     ...      ...     ...      ...     ...           ...   
 2447    2023     146     1361      57     1166      56  San Diego St   
 2448    2023     152     1163      72     1274      59   Connecticut   
 2449    2023     152     1361      72     1194      71  San Diego St   
 2450    2023     154     1163      76     1361      59   Connecticut   
 2451    2021     138     1179       1     1455       0        Oregon   
 
              Lteam  Round Wseed Region  RegionNum Lseed  
 0             Iowa      1   X09      X          2   X08  
 1  

In [17]:
## there was a no contest in 2022 so we have to add vcu and Oregon into the teams
tourn2021=tourn_region[(tourn_region.Season==2021)]
tourn2021=tourn_region[(tourn_region.Season==2021)]
for i in tourn2021.Round.unique():
    print(i)
    display(tourn2021[tourn2021.Round==i].shape)


136


(4, 13)

137


(16, 13)

138


...

(2, 13)

154


(1, 13)

1


(1, 13)

In [18]:

tourn_region['RegionNum']=tourn_region['RegionNum'].astype(int)

tourn_region['Wseed'].str.replace("[a-zA-Z]","",regex=True).astype(int)

0        9
1       11
2        1
3        9
4        3
        ..
2447     5
2448     4
2449     5
2450     4
2451     7
Name: Wseed, Length: 2452, dtype: int32

In [19]:

tourn_region['RegionNum']=tourn_region['RegionNum'].astype(int)
Wseed=(tourn_region['Wseed'].str.replace("[a-zA-Z]","",regex=True)).astype(int)
Lseed=(tourn_region['Lseed'].str.replace("[a-zA-Z]","",regex=True)).astype(int)
real_seeds=tourn_region[['Wseed','Lseed']].values
choose_seeds=np.concatenate([Wseed.values.reshape(-1,1),Lseed.values.reshape(-1,1)],axis=1)


In [20]:
teams=tourn_region[['Wteam','Lteam']].values


for i in range(len(choose_seeds)):
    g=choose_seeds[i,:]
    if g[0]==g[1]:
        choose_seeds[i,0]+=1
        
high_arg=np.argmax(choose_seeds,1).reshape(-1,1)
low_arg=np.argmin(choose_seeds,1).reshape(-1,1)
tourn_region['high_team']=''
tourn_region['high_seed']=np.nan
tourn_region['low_team']=''

tourn_region['low_seed']=np.nan

tourn_region.loc[tourn_region.index,'high_team']=np.take_along_axis(teams,high_arg,1)
tourn_region.loc[tourn_region.index,'high_seed']=np.take_along_axis(real_seeds,high_arg,1)
tourn_region.loc[tourn_region.index,'low_team']=np.take_along_axis(teams,low_arg,1)
tourn_region.loc[tourn_region.index,'low_seed']=np.take_along_axis(real_seeds,low_arg,1)
tourn_region=tourn_region.rename(columns={'Season':'Year'})
tourn_region['Region']=tourn_region['Region'].replace(region_to_region)


In [21]:
tourn_region

,Year,Daynum,Wteamid,Wscore,Lteamid,Lscore,Wteam,Lteam,Round,Wseed,Region,RegionNum,Lseed,high_team,high_seed,low_team,low_seed
0,1985,136,1116,63,1234,54,Arkansas,Iowa,1,X09,Midwest,2,X08,Arkansas,X09,Iowa,X08
1,1985,136,1120,59,1345,58,Auburn,Purdue,1,Z11,West,4,Z06,Auburn,Z11,Purdue,Z06
2,1985,136,1207,68,1250,43,Georgetown,Lehigh,1,W01,East,1,W16,Lehigh,W16,Georgetown,W01
3,1985,136,1229,58,1425,55,Illinois St,USC,1,Y09,South,3,Y08,Illinois St,Y09,USC,Y08
4,1985,136,1242,49,1325,38,Kansas,Ohio,1,Z03,West,4,Z14,Ohio,Z14,Kansas,Z03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447,2023,146,1361,57,1166,56,San Diego St,Creighton,4,X05,Midwest,2,X06,Creighton,X06,San Diego St,X05
2448,2023,152,1163,72,1274,59,Connecticut,Miami FL,5,Z04,West,4,Y05,Miami FL,Y05,Connecticut,Z04
2449,2023,152,1361,72,1194,71,San Diego St,FL Atlantic,5,X05,Midwest,2,W09,FL Atlantic,W09,San Diego St,X05
2450,2023,154,1163,76,1361,59,Connecticut,San Diego St,6,Z04,West,4,X05,San Diego St,X05,Connecticut,Z04


In [22]:
base_columns = ['Year','Round','Daynum','Region','RegionNum']
team0_columns = ['high_team','high_seed']
team1_columns = ['low_team','low_seed']
rename_columns=['Team','Seed']

In [23]:
high_seed_df = tourn_region[base_columns + team0_columns].rename(columns=dict(zip(team0_columns,rename_columns))).copy()
low_seed_df  = tourn_region[base_columns + team1_columns].rename(columns=dict(zip(team1_columns,rename_columns))).copy()

high_seed_df['Team_Position'] = 'high'
low_seed_df['Team_Position'] = 'low'
raw_long_df = pd.concat([high_seed_df, low_seed_df],axis=0)
raw_long_df ['Mapped_Round']=raw_long_df ['Round']
raw_long_df=raw_long_df.drop('Round',axis=1)
display(double_rounds)
raw_long_df


,Seed,GameRound,GameSlot,Daynum,LateDayNum,EarlyDayNum
0,W01,1,R1W1,136,137.0,NaN
1,W01,2,R2W1,138,139.0,NaN
2,W01,3,R3W1,143,144.0,NaN
3,W01,4,R4W1,145,146.0,NaN
4,W01,5,R5WX,152,152.0,NaN
...,...,...,...,...,...,...
715,Z16b,2,R2Z1,139,NaN,138.0
716,Z16b,3,R3Z1,144,NaN,143.0
717,Z16b,4,R4Z1,146,NaN,145.0
718,Z16b,5,R5YZ,152,NaN,152.0


,Year,Daynum,Region,RegionNum,Team,Seed,Team_Position,Mapped_Round
0,1985,136,Midwest,2,Arkansas,X09,high,1
1,1985,136,West,4,Auburn,Z11,high,1
2,1985,136,East,1,Lehigh,W16,high,1
3,1985,136,South,3,Illinois St,Y09,high,1
4,1985,136,West,4,Ohio,Z14,high,1
...,...,...,...,...,...,...,...,...
2447,2023,146,Midwest,2,San Diego St,X05,low,4
2448,2023,152,West,4,Connecticut,Z04,low,5
2449,2023,152,Midwest,2,San Diego St,X05,low,5
2450,2023,154,West,4,Connecticut,Z04,low,6


In [24]:
tourn_with_rounds=pd.merge(raw_long_df,double_rounds,left_on=['Daynum','Seed'],right_on=['Daynum','Seed'])
tourn_with_rounds.Seed.isna().sum()

0

In [25]:
new_year=seed_df[seed_df.Season==2023]
new_year=new_year.rename(columns={'TeamName':'Team','Season':'Year'})
new_year['Seed']=new_year['Seed']#.str.replace("[a-zA-Z]","",regex=True).astype(int)
new_year['GameRound']=1


In [26]:
long_tourn =tourn_with_rounds.reset_index().copy()
y2021idx=long_tourn[(long_tourn.Year==2019)].index
# long_tourn .loc[y2021idx,'GameRound']=long_tourn .loc[y2021idx,'Mapped_Round']

y2022=long_tourn .loc[y2021idx,['Daynum','GameRound']]
y2022.value_counts().sort_index(),y2022.shape,y2022.value_counts().sum()
# long_tourn.loc[y2021idx]

(Daynum  GameRound
 134     0             4
 135     0             4
 136     1            32
 137     1            32
 138     2            16
 139     2            16
 143     3             8
 144     3             8
 145     4             4
 146     4             4
 152     5             8
 154     6             4
 Name: count, dtype: int64,
 (140, 2),
 140)

In [27]:


long_tourn =tourn_with_rounds.reset_index().copy()
y2021idx=long_tourn[(long_tourn.Year==2022)].index
# long_tourn .loc[y2021idx,'GameRound']=long_tourn .loc[y2021idx,'Mapped_Round']

y2022=long_tourn .loc[y2021idx,['Daynum','GameRound']]
y2022.value_counts().sort_index(),y2022.shape,y2022.value_counts().sum()
# long_tourn.loc[y2021idx]

(Daynum  GameRound
 134     0             4
 135     0             4
 136     1            32
 137     1            32
 138     2            16
 139     2            16
 143     3             8
 144     3             8
 145     4             4
 146     4             4
 152     5             8
 154     6             4
 Name: count, dtype: int64,
 (140, 2),
 140)

In [28]:

long_tourn =tourn_with_rounds.reset_index().copy()
y2021idx=long_tourn[(long_tourn.Year==2021)].index
# long_tourn .loc[y2021idx,'GameRound']=long_tourn .loc[y2021idx,'Mapped_Round']

y2021=long_tourn .loc[y2021idx,['Daynum','GameRound']]
y2021.value_counts().sort_index(),y2021.shape,y2021.value_counts().sum()

(Daynum  GameRound
 136     1             8
 137     1            32
 138     2            32
 139     2            16
 145     4             8
 146     4             8
 152     5             8
 154     6             4
 Name: count, dtype: int64,
 (116, 2),
 116)

In [29]:
#change the day num to round
round_map_2021={
         ## first 4
          136:0,## first 4

          ## first round
          137:1,
          138:1,

          # # second round

          139:2,
          
          # # sweet 16
          145:3,
          146:3,

          # # elite 8        
          147:4,
          148:4,            
          # # final 4  
          152:5,
          # # chamionship 
          154:6}

In [30]:
last_year=long_tourn[long_tourn.Year==2022].copy()
last_year

last_year.loc[last_year.index,'Team']=['TBD' for c in range(len(last_year))]
last_year.loc[last_year.index,'Seed']=['64' for c in range(len(last_year))]
last_year.loc[last_year.index,'Year']=[2023 for c in range(len(last_year))]


last_year=last_year[last_year.GameRound>1]
last_year
long_tourn=pd.concat([long_tourn,new_year,last_year],axis=0,ignore_index=True)
long_tourn=long_tourn.rename(columns={'GameRound':'Round'})

long_tourn['Seed']=(long_tourn['Seed'].str.replace("[a-zA-Z]","",regex=True)).astype(int)
tourn_region['Seed']=(tourn_region['Wseed'].str.replace("[a-zA-Z]","",regex=True)).astype(int)
long_tourn=long_tourn.drop(
  ['LateDayNum','EarlyDayNum','TeamID','Region_'],axis=1
).sort_values(['Year','Round','Seed'])
long_tourn

,index,Year,Daynum,Region,RegionNum,Team,Seed,Team_Position,Mapped_Round,Round,GameSlot
3059,3059.0,1985,136.0,East,1,Georgetown,1,low,1.0,1,R1W1
3241,3241.0,1985,136.0,South,3,Oklahoma,1,low,1.0,1,R1Y1
3282,3282.0,1985,136.0,Midwest,2,St John's,1,low,1.0,1,R1X1
3517,3517.0,1985,137.0,West,4,Michigan,1,low,1.0,1,R1Z1
3184,3184.0,1985,136.0,West,4,North Carolina,2,low,1.0,1,R1Z2
...,...,...,...,...,...,...,...,...,...,...,...
3018,3018.0,2023,154.0,West,4,San Diego St,5,high,6.0,6,R6CH
5211,2777.0,2023,154.0,South,3,TBD,64,low,6.0,6,R6CH
5212,2778.0,2023,154.0,South,3,TBD,64,low,6.0,6,R6CH
5220,3004.0,2023,154.0,South,3,TBD,64,high,6.0,6,R6CH


In [31]:
final_four_games = \
long_tourn[long_tourn['Round'] == 4] \
    .assign(game_num = lambda x: (x['RegionNum'] <= 2)+1) \
    .groupby(['Year','game_num']) \
    .agg({'Region' : lambda x: dict(zip(['team1','team2'],x))}) \
    .groupby('Year') \
    .agg({'Region' : lambda x: dict(zip(['game1','game2'],x))}) \
    .to_dict().get('Region')

In [32]:
years = long_tourn.Year.unique().tolist()

brackets_dict = {}
for year in years:

    that_year_df = long_tourn[long_tourn['Year'] == year].copy()
    that_year_win = tourn_region[tourn_region['Year'] == year].copy()
    
    round0= that_year_df[that_year_df['Round'] == 0].sort_values('Seed')

    round1 = that_year_df[that_year_df['Round'] == 1].sort_values('Seed')
    round2 = that_year_df[that_year_df['Round'] == 2]
    round3 = that_year_df[that_year_df['Round'] == 3]
    round4 = that_year_df[that_year_df['Round'] == 4]
    round5 = that_year_df[that_year_df['Round'] == 5]
    round6 = that_year_df[that_year_df['Round'] == 6]

    print(that_year_df['Round'].unique())
    round6_win = that_year_win[that_year_win['Round'] == 6]
    round6_win=round6_win.rename(columns={'Wteam':'Team','Wseed':'Seed'})

    try:
        winner = round6_win[['Team','Seed']].reset_index(drop=True).T.to_dict()[0]

    except:
        
        
        winner={'Team': 'TBD', 'Seed': 0}
    # print(winner)
    
    east1 = round1[round1['Region'] == 'W']
    west1 = round1[round1['Region'] == 'X']
    midwest1 = round1[round1['Region'] == 'Y']
    south1 = round1[round1['Region'] == 'Z']

    regs={
            'W':east1,
            'X':west1,
            'Y':midwest1,
            'Z':south1,
                }

    print(year,len(round1))
    if i !=2021:
        for r in regs:
            if len(regs[r])!=16:
                print(round1['Region'].unique())
                display(r,regs[r])
                regs[r]=regs[r].drop_duplicates('Seed',keep='first').reset_index(drop=True)
                display(r,regs[r])



    results = {
        # 'playin' : round0[['Team','Seed']].to_dict('records'),
        'first' : round1[['Team','Seed']].to_dict('records'),
        'second' : round2[['Team','Seed']].to_dict('records'),
        'sweet16' : round3[['Team','Seed']].to_dict('records'),
        'elite8' : round4[['Team','Seed']].to_dict('records'),
        'final4' : round5[['Team','Seed']].to_dict('records'),
        'championship' : round6[['Team','Seed']].to_dict('records'),
        'winner' : {'Team' : winner['Team'], 'Seed' : int(winner['Seed'])}
    }
    regions = {
        'W' : regs['W'][['Team','Seed']].to_dict('records'),
        'X' : regs['X'][['Team','Seed']].to_dict('records'),
        'Y' : regs['Y'][['Team','Seed']].to_dict('records'),
        'Z' : regs['Z'][['Team','Seed']].to_dict('records')
    }
    brackets_dict[year] = {'Region' : regions, 'Results': results, 'Finals': final_four_games[year]}

[1 2 3 4 5 6]
1985 64
['East' 'South' 'Midwest' 'West']


'W'

,index,Year,Daynum,Region,RegionNum,Team,Seed,Team_Position,Mapped_Round,Round,GameSlot


'W'

,index,Year,Daynum,Region,RegionNum,Team,Seed,Team_Position,Mapped_Round,Round,GameSlot


...

['South' 'Midwest' 'West' 'East' 'W' 'X' 'Y' 'Z']


'Z'

,index,Year,Daynum,Region,RegionNum,Team,Seed,Team_Position,Mapped_Round,Round,GameSlot
5159,NaN,2023,NaN,Z,4,Kansas,1,NaN,NaN,1,NaN
5160,NaN,2023,NaN,Z,4,UCLA,2,NaN,NaN,1,NaN
5161,NaN,2023,NaN,Z,4,Gonzaga,3,NaN,NaN,1,NaN
5162,NaN,2023,NaN,Z,4,Connecticut,4,NaN,NaN,1,NaN
5163,NaN,2023,NaN,Z,4,St Mary's CA,5,NaN,NaN,1,NaN
5164,NaN,2023,NaN,Z,4,TCU,6,NaN,NaN,1,NaN
5165,NaN,2023,NaN,Z,4,Northwestern,7,NaN,NaN,1,NaN
5166,NaN,2023,NaN,Z,4,Arkansas,8,NaN,NaN,1,NaN
5167,NaN,2023,NaN,Z,4,Illinois,9,NaN,NaN,1,NaN
5168,NaN,2023,NaN,Z,4,Boise St,10,NaN,NaN,1,NaN


'Z'

,index,Year,Daynum,Region,RegionNum,Team,Seed,Team_Position,Mapped_Round,Round,GameSlot
0,NaN,2023,NaN,Z,4,Kansas,1,NaN,NaN,1,NaN
1,NaN,2023,NaN,Z,4,UCLA,2,NaN,NaN,1,NaN
2,NaN,2023,NaN,Z,4,Gonzaga,3,NaN,NaN,1,NaN
3,NaN,2023,NaN,Z,4,Connecticut,4,NaN,NaN,1,NaN
4,NaN,2023,NaN,Z,4,St Mary's CA,5,NaN,NaN,1,NaN
5,NaN,2023,NaN,Z,4,TCU,6,NaN,NaN,1,NaN
6,NaN,2023,NaN,Z,4,Northwestern,7,NaN,NaN,1,NaN
7,NaN,2023,NaN,Z,4,Arkansas,8,NaN,NaN,1,NaN
8,NaN,2023,NaN,Z,4,Illinois,9,NaN,NaN,1,NaN
9,NaN,2023,NaN,Z,4,Boise St,10,NaN,NaN,1,NaN


In [33]:
import json
with open(r'\Users\sebas\IDrive-Sync\MetaLocalLLC\RL-Bots\Tournament-gym\bracketology_rl\data\Mbrackets_2023.json' , 'w') as f:
    json.dump(brackets_dict, f)

: 